# Python for Finance - 3 - CAPM (Capital Asset Pricing Model)

* Author: Cleiber Garcia
* Date: November/2023

* Goal: I taught myself how to use Python with financial data for making predictions. I put together this notebook while taking an online course called Python para Finanças ('Python for Finance') on Udemy, taught by Jones Granatyr (from IA Expert Academy). This notebook closely mirrors the one covered in class, with the effort I made to infuse my personal touch whenever possible and appropriate. I assure you that I meticulously wrote it line by line.

* Disclaimer: **Please be advised that all forecasts and patterns developed in the course of this training are 
intended solely for educational purposes. Under no circumstances should they be construed as 
investment advice.**

For more information please contact me at cleiber.garcia@gmail.com

# Summary
* [1. Initial Setup](#InitialSetup)
* [2. Calculating Beta for an Asset](#CalculatingBetaOneAsset)
* [3. Calculating Beta for All Assets](#BetaForAllAssets)
* [4. Calculating CAPM for the Portfolio 'AMBEV', 'ODONTOPREV', ... ](#CapmPortfolioAmbev)

# 1. Initial Setup <a id='InitialSetup'></a>

## 1.1 Importing Python Modules

In [22]:
import pandas as pd
import numpy as np
import plotly.express as px

## 1.2 Loading Working Dataset

In [23]:
dataset = pd.read_csv('acoes_new.csv')
dataset

,Date,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,2015-01-02,14.99,15.20,5.923076,0.232812,11.910702,47.259998
1,2015-01-05,14.85,15.00,5.963461,0.237187,11.544731,46.320000
2,2015-01-06,15.21,14.80,5.875000,0.234062,10.822770,46.580002
3,2015-01-07,14.55,14.67,5.807692,0.241875,10.746248,48.150002
4,2015-01-08,14.27,14.15,5.905769,0.240000,10.995774,48.509998
...,...,...,...,...,...,...,...
2164,2023-10-24,7.48,2.88,35.009998,1.410000,25.600000,110.239998
2165,2023-10-25,7.60,2.90,31.469999,1.440000,25.500000,109.349998
2166,2023-10-26,8.23,2.96,32.660000,1.470000,26.400000,111.400002
2167,2023-10-27,7.77,2.85,31.629999,1.460000,25.559999,109.849998


In [24]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2169 entries, 0 to 2168
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2169 non-null   object 
 1   GOL     2169 non-null   float64
 2   CVC     2169 non-null   float64
 3   WEGE    2169 non-null   float64
 4   MGLU    2169 non-null   float64
 5   TOTS    2169 non-null   float64
 6   BOVA    2169 non-null   float64
dtypes: float64(6), object(1)
memory usage: 118.7+ KB


In [25]:
dataset.describe()

,GOL,CVC,WEGE,MGLU,TOTS,BOVA
count,2169.000000,2169.000000,2169.000000,2169.000000,2169.000000,2169.000000
mean,14.534905,25.375650,19.058307,6.453758,18.964800,84.371761
std,9.148261,16.903947,13.416379,7.378103,9.464138,24.255406
min,1.160000,2.200000,4.942307,0.030585,7.156402,36.450001
25%,7.580000,13.549243,6.957692,0.712343,10.193964,61.900002
50%,12.280000,20.080000,10.585000,3.700000,14.573333,91.160004
75%,20.240000,40.599998,33.930000,8.997500,28.049999,105.949997
max,43.790001,64.800003,46.105000,27.450001,40.599998,125.750000


In [26]:
dataset_bkp = dataset.copy()

In [27]:
# Deleting the Date column
dataset.drop(labels = ['Date'], axis = 1, inplace = True)

In [28]:
# Normalizing the dataset
dataset_normalizado = dataset.copy()
for i in dataset.columns:
    dataset_normalizado[i] = dataset[i] / dataset[i][0]
dataset_normalizado

,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.990660,0.986842,1.006818,1.018792,0.969274,0.980110
2,1.014676,0.973684,0.991883,1.005369,0.908659,0.985612
3,0.970647,0.965132,0.980520,1.038928,0.902235,1.018832
4,0.951968,0.930921,0.997078,1.030875,0.923184,1.026449
...,...,...,...,...,...,...
2164,0.498999,0.189474,5.910780,6.056389,2.149328,2.332628
2165,0.507005,0.190789,5.313117,6.185248,2.140932,2.313796
2166,0.549033,0.194737,5.514027,6.314108,2.216494,2.357173
2167,0.518346,0.187500,5.340130,6.271155,2.145969,2.324376


In [29]:
# Calculating the return rate
dataset_taxa_retorno = (dataset_normalizado / dataset_normalizado.shift(1)) - 1
dataset_taxa_retorno

,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.009340,-0.013158,0.006818,0.018792,-0.030726,-0.019890
2,0.024242,-0.013333,-0.014834,-0.013175,-0.062536,0.005613
3,-0.043392,-0.008784,-0.011457,0.033380,-0.007070,0.033705
4,-0.019244,-0.035447,0.016887,-0.007752,0.023220,0.007477
...,...,...,...,...,...,...
2164,0.027473,0.054945,0.032743,-0.066225,-0.005825,0.008139
2165,0.016043,0.006944,-0.101114,0.021277,-0.003906,-0.008073
2166,0.082895,0.020690,0.037814,0.020833,0.035294,0.018747
2167,-0.055893,-0.037162,-0.031537,-0.006803,-0.031818,-0.013914


In [30]:
# Count the number of null values
dataset_taxa_retorno.isnull().sum()

GOL     1
CVC     1
WEGE    1
MGLU    1
TOTS    1
BOVA    1
dtype: int64

In [33]:
# Preenche com zero a primeira linha (substitui NaN por 0)
dataset_taxa_retorno.fillna(0, inplace=True)
dataset_taxa_retorno

,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.009340,-0.013158,0.006818,0.018792,-0.030726,-0.019890
2,0.024242,-0.013333,-0.014834,-0.013175,-0.062536,0.005613
3,-0.043392,-0.008784,-0.011457,0.033380,-0.007070,0.033705
4,-0.019244,-0.035447,0.016887,-0.007752,0.023220,0.007477
...,...,...,...,...,...,...
2164,0.027473,0.054945,0.032743,-0.066225,-0.005825,0.008139
2165,0.016043,0.006944,-0.101114,0.021277,-0.003906,-0.008073
2166,0.082895,0.020690,0.037814,0.020833,0.035294,0.018747
2167,-0.055893,-0.037162,-0.031537,-0.006803,-0.031818,-0.013914


In [59]:
# Anualização da taxa média de retorno
dataset_taxa_retorno.mean() * 246

GOL     0.209138
CVC    -0.005317
WEGE    0.251358
MGLU    0.422210
TOTS    0.158701
BOVA    0.126699
dtype: float64

[Summary](#Summary)

# 2. Calculating Beta for an Asset <a id='CalculatingBetaOneAsset'></a>

## 2.1 Calculating Beta using Linear Regression

In [36]:
figura = px.scatter(dataset_taxa_retorno, x='BOVA', y='MGLU', title='Return Rate for BOVA X MGLU')
figura.show()

In [40]:
beta, alpha = np.polyfit(x=dataset_taxa_retorno['BOVA'], y=dataset_taxa_retorno['MGLU'], deg=1)
print(f'beta: {beta}')
print(f'alpha: {alpha}')
print(f'alpha (%): {alpha * 100}%')

beta: 1.2242983977719688
alpha: 0.0010857433718713914
alpha (%): 0.10857433718713914%


In [41]:
# beta * ??? + alpha

In [43]:
figura = px.scatter(dataset_taxa_retorno, x = 'BOVA', y = 'MGLU', title='Return Rate for BOVA X MGLU')
figura.add_scatter(x = dataset_taxa_retorno['BOVA'], y = beta * dataset_taxa_retorno['BOVA'] + alpha)
figura.show()

## 2.2 Calculating Beta using Covariance and Variance

In [45]:
dataset_taxa_retorno

,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.009340,-0.013158,0.006818,0.018792,-0.030726,-0.019890
2,0.024242,-0.013333,-0.014834,-0.013175,-0.062536,0.005613
3,-0.043392,-0.008784,-0.011457,0.033380,-0.007070,0.033705
4,-0.019244,-0.035447,0.016887,-0.007752,0.023220,0.007477
...,...,...,...,...,...,...
2164,0.027473,0.054945,0.032743,-0.066225,-0.005825,0.008139
2165,0.016043,0.006944,-0.101114,0.021277,-0.003906,-0.008073
2166,0.082895,0.020690,0.037814,0.020833,0.035294,0.018747
2167,-0.055893,-0.037162,-0.031537,-0.006803,-0.031818,-0.013914


In [46]:
dataset_taxa_retorno_bkp = dataset_taxa_retorno

In [47]:
# Calculating the covariance matrix
matriz_covariancia = dataset_taxa_retorno.drop(columns = ['GOL', 'CVC', 'WEGE', 'TOTS']).cov() * 246
matriz_covariancia

,MGLU,BOVA
MGLU,0.449651,0.077639
BOVA,0.077639,0.063415


In [48]:
cov_mglu_bova = matriz_covariancia.iloc[1, 0]
cov_mglu_bova

0.07763871951533598

In [49]:
# Calculating the variance for 'BOVA'
variancia_bova = dataset_taxa_retorno['BOVA'].var() * 246
variancia_bova

0.06341486655265278

In [50]:
# Calculating Beta for 'MGLU'
beta_mglu = cov_mglu_bova / variancia_bova
beta_mglu

1.2242983977719681

In [52]:
# Calculating the annual average return for 'BOVA'
rm = dataset_taxa_retorno['BOVA'].mean() * 246
rm

0.12669887261190632

In [54]:
taxa_selic_2015 = 14.25
taxa_selic_2016 = 13.75
taxa_selic_2017 = 7.00
taxa_selic_2018 = 6.50
taxa_selic_2019 = 4.50
taxa_selic_2020 = 2.00
taxa_selic_2021 = 9.25
taxa_selic_2022 = 13.75
taxa_selic_2023 = 12.75

In [55]:
# Calculating the historical selic tax
taxa_selic_historico = np.array([14.25, 13.75, 7.00, 6.50, 4.50, 2.00, 9.25, 13.75, 12.75])

In [56]:
# Calculating the risk free return
rf = taxa_selic_historico.mean() / 100
rf

0.09305555555555556

In [58]:
# Calculating CAPM for 'MGLU'
capm_mglu = rf + (beta * (rm - rf))
capm_mglu

0.13424501472338016

[Summary](#Summary)

# 3. Calculating Beta for All Assets <a id='BetaForAllAssets'></a>

## 3.1 Calculating Alpha and Beta using Linear Regression

In [60]:
dataset_taxa_retorno.head()

,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.009340,-0.013158,0.006818,0.018792,-0.030726,-0.019890
2,0.024242,-0.013333,-0.014834,-0.013175,-0.062536,0.005613
3,-0.043392,-0.008784,-0.011457,0.033380,-0.007070,0.033705
4,-0.019244,-0.035447,0.016887,-0.007752,0.023220,0.007477


In [61]:
# Calculating Alpha and Beta using Linear Regression
betas = []
alphas = []
for ativo in dataset_taxa_retorno.columns[0:-1]:
    #print(ativo) 
    beta, alpha = np.polyfit(dataset_taxa_retorno['BOVA'], dataset_taxa_retorno[ativo], 1)
    betas.append(beta)
    alphas.append(alpha)

In [62]:
betas

[1.7612540371425498,
 1.286426487893619,
 0.7029324954463638,
 1.2242983977719688,
 0.7663493019788943]

In [63]:
alphas

[-5.6953969367397585e-05,
 -0.0006841696965959104,
 0.0006597432014532551,
 0.0010857433718713914,
 0.00025042834367582263]

In [64]:
def visualiza_betas_alphas(betas, alphas):
    for i, ativo in enumerate(dataset_taxa_retorno.columns[0:-1]):
        #print(i, ativo)
        print(ativo, 'beta:', betas[i], 'alpha:', alphas[i] * 100)

In [65]:
visualiza_betas_alphas(betas, alphas)

GOL beta: 1.7612540371425498 alpha: -0.005695396936739759
CVC beta: 1.286426487893619 alpha: -0.06841696965959104
WEGE beta: 0.7029324954463638 alpha: 0.0659743201453255
MGLU beta: 1.2242983977719688 alpha: 0.10857433718713914
TOTS beta: 0.7663493019788943 alpha: 0.025042834367582263


In [66]:
np.array(alphas).mean() * 100

0.025095825020743225

## 3.2 Calculating CAPM for the Portfolio

In [76]:
# Calculating the risk free return
rf = taxa_selic_historico.mean() / 100
rf

0.09305555555555556

In [75]:
# Calculating the annual average return for 'BOVA'
rm = dataset_taxa_retorno['BOVA'].mean() * 246
rm

0.12669887261190632

In [70]:
# Calculating the CAPM for the various Assets
capm_empresas = []
for i, ativo in enumerate(dataset_taxa_retorno.columns[0:-1]):
    #print(i, ativo)
    capm_empresas.append(rf + (betas[i] * (rm - rf)))
    
capm_empresas

[0.15230998354392014,
 0.13633520975744834,
 0.11670453636906941,
 0.13424501472338016,
 0.11883808809794459]

In [71]:
def visualiza_capm(capm):
    for i, ativo in enumerate(dataset_taxa_retorno.columns[0:-1]):
        print(ativo, 'CAPM:', capm[i] * 100)

In [72]:
# A GOL apresentou retorno de 15.23% acima do mercado
visualiza_capm(capm_empresas)

GOL CAPM: 15.230998354392014
CVC CAPM: 13.633520975744833
WEGE CAPM: 11.670453636906942
MGLU CAPM: 13.424501472338015
TOTS CAPM: 11.88380880979446


In [73]:
# Set weights for the CAPM according to the Asset
pesos = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

In [74]:
# Se investirmos nesse portfolio, teremos um retorno de 13.17% acima do mercado, 
# para compensar o risco que corremos
capm_portfolio = np.sum(capm_empresas * pesos) * 100
capm_portfolio

13.168656649835253

[Summary](#Summary)

# 4. Calculating CAPM for the Portfolio 'AMBEV', 'ODONTOPREV', ... <a id='CapmPortfolioAmbev'></a>

In [133]:
# Loading the working dataset
dataset = pd.read_csv('acoes_ex.csv')
dataset.drop(labels = ['Date'], axis = 1, inplace = True)
dataset_normalizado = dataset.copy()

In [134]:
# Normalizing the dataset
for i in dataset.columns:
    dataset_normalizado[i] = dataset[i] / dataset[i][0]

In [135]:
dataset_normalizado

,AMBEV,ODONTOPREV,VIVO,PETROBRAS,BBRASIL,BOVA
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.981898,1.026681,0.980169,0.918889,0.979249,0.980110
2,1.019351,1.019210,0.955844,0.895556,0.992936,0.985612
3,1.033708,1.034152,0.988630,0.938889,1.036645,1.018832
4,1.032459,1.030950,1.028821,1.002222,1.040177,1.026449
...,...,...,...,...,...,...
2188,0.781523,1.107791,1.210471,4.584444,2.185431,2.340246
2189,0.779650,1.143010,1.193548,4.534445,2.178366,2.316970
2190,0.779026,1.178228,1.191962,4.261111,2.175276,2.313796
2191,0.781523,1.196371,1.211528,4.326667,2.165121,2.332628


In [136]:
# Calculating the daily return
dataset_taxa_retorno = (dataset_normalizado / dataset_normalizado.shift(1)) - 1
dataset_taxa_retorno

,AMBEV,ODONTOPREV,VIVO,PETROBRAS,BBRASIL,BOVA
0,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.018102,0.026681,-0.019831,-0.081111,-0.020751,-0.019890
2,0.038144,-0.007276,-0.024818,-0.025393,0.013977,0.005613
3,0.014084,0.014660,0.034301,0.048387,0.044020,0.033705
4,-0.001208,-0.003096,0.040653,0.067456,0.003407,0.007477
...,...,...,...,...,...,...
2188,-0.000798,-0.009542,-0.001309,-0.007219,0.007736,0.002084
2189,-0.002396,0.031792,-0.013980,-0.010906,-0.003232,-0.009946
2190,-0.000801,0.030812,-0.001329,-0.060279,-0.001419,-0.001370
2191,0.003205,0.015399,0.016415,0.015385,-0.004668,0.008139


In [137]:
# Set the first row with zeros (instead of NaN)
dataset_taxa_retorno.fillna(0, inplace=True)
dataset_taxa_retorno

,AMBEV,ODONTOPREV,VIVO,PETROBRAS,BBRASIL,BOVA
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.018102,0.026681,-0.019831,-0.081111,-0.020751,-0.019890
2,0.038144,-0.007276,-0.024818,-0.025393,0.013977,0.005613
3,0.014084,0.014660,0.034301,0.048387,0.044020,0.033705
4,-0.001208,-0.003096,0.040653,0.067456,0.003407,0.007477
...,...,...,...,...,...,...
2188,-0.000798,-0.009542,-0.001309,-0.007219,0.007736,0.002084
2189,-0.002396,0.031792,-0.013980,-0.010906,-0.003232,-0.009946
2190,-0.000801,0.030812,-0.001329,-0.060279,-0.001419,-0.001370
2191,0.003205,0.015399,0.016415,0.015385,-0.004668,0.008139


In [138]:
# Calculating Alpha and Beta using Linear Regression
betas = []
alphas = []
for ativo in dataset_taxa_retorno.columns[0:-1]:
    #print(ativo) 
    beta, alpha = np.polyfit(dataset_taxa_retorno['BOVA'], dataset_taxa_retorno[ativo], 1)
    betas.append(beta)
    alphas.append(alpha)

In [139]:
betas

[0.5980149752861414,
 0.4512386743057557,
 0.500682075026074,
 1.4409275423907746,
 1.3033274878549699]

In [140]:
alphas

[-0.00027128463610177786,
 3.6704061519110586e-05,
 9.608028790772638e-08,
 0.00044751411629568455,
 5.6968309358945545e-05]

In [141]:
# Retorno BBRASIL é 30.33% mais volátil que o mercado
# O retorno da PETROBRÁS excedeu o mercado em 4.475%
# (excedeu o indice 'BOVA' em 4.475%)
# O retorno da AMBEV ficou 2.71% abaixo do mercado
visualiza_betas_alphas(betas, alphas)

AMBEV beta: 0.5980149752861414 alpha: -0.027128463610177785
ODONTOPREV beta: 0.4512386743057557 alpha: 0.0036704061519110586
VIVO beta: 0.500682075026074 alpha: 9.608028790772638e-06
PETROBRAS beta: 1.4409275423907746 alpha: 0.04475141162956846
BBRASIL beta: 1.3033274878549699 alpha: 0.0056968309358945545


In [142]:
# Calculating the average return
# (Excedeu a BOVA em 0.005% em média)
np.array(alphas).mean() * 100

0.00539995862719741

In [143]:
taxa_selic_2015 = 14.25
taxa_selic_2016 = 13.75
taxa_selic_2017 = 7.00
taxa_selic_2018 = 6.50
taxa_selic_2019 = 4.50
taxa_selic_2020 = 2.00
taxa_selic_2021 = 9.25
taxa_selic_2022 = 13.75
taxa_selic_2023 = 12.75

In [144]:
# Calculating the historical selic tax
taxa_selic_historico = np.array([14.25, 13.75, 7.00, 6.50, 4.50, 2.00, 9.25, 13.75, 12.75])

In [145]:
# Calculating the risk free return
rf = taxa_selic_historico.mean() / 100
rf

0.09305555555555556

In [146]:
# Calculating the annual average return for 'BOVA'
rm = dataset_taxa_retorno['BOVA'].mean() * 246
rm

0.12398155067338709

In [147]:
# Calculating the CAPM for the various Assets
capm_empresas = []
for i, ativo in enumerate(dataset_taxa_retorno.columns[0:-1]):
    #print(i, ativo)
    capm_empresas.append(rf + (betas[i] * (rm - rf)))

In [148]:
# Se investirmos na PETROBRÁS, ganharemos 13.76% para compensar o 
# risco que corremos
visualiza_capm(capm_empresas)

AMBEV CAPM: 11.154976376164491
ODONTOPREV CAPM: 10.701056059411213
VIVO CAPM: 10.85396469633977
PETROBRAS CAPM: 13.761767369668165
BBRASIL CAPM: 13.3362255081894


In [149]:
# Set weights for the CAPM for each Asset
pesos = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

In [150]:
# Se investirmos no portfólio, ganharemos 11.96% de retorno para ser 
# compensado pelo risco que corremos
capm_portfólio = np.sum(capm_empresas * pesos) * 100
capm_portfólio

11.961598001954608